In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from time import sleep
import os
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation

2.3.0


In [2]:
classes = [
    'Doing other things',
    'Swiping Right',
    'Swiping Left',
    'Zooming In With Two Fingers',
    'Zooming Out With Two Fingers',
    'Stop Sign',
    'No gesture'
]

In [4]:
classes[1]

'Swiping Right'

In [5]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [6]:
def normaliz_data(np_data):
    # Normalisation
    scaler = StandardScaler()
    #scaled_images  = normaliz_data2(np_data)
    scaled_images  = np_data.reshape(-1, 30, 64, 64, 1)
    return scaled_images

In [7]:
def normaliz_data2(v):
    normalized_v = v / np.sqrt(np.sum(v**2))
    return normalized_v

In [8]:
class Conv3DModel(tf.keras.Model):
    def __init__(self):
        super(Conv3DModel, self).__init__()
        self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
        self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')
        self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
    #norm
        self.flatten =  tf.keras.layers.Flatten(name="flatten")

    # Dense layers
        self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
        self.out = tf.keras.layers.Dense(7, activation='softmax', name="output")
        
        
    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.convLSTM(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.out(x)

In [9]:
model1 = Conv3DModel()
model1.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.RMSprop())

In [10]:
model2 = Conv3DModel()
model2.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.RMSprop())

In [13]:
model1.load_weights('D:/3D-CNN/D/FinalModel/3D-CNN_LSTM_weights')

In [14]:
model2.load_weights('D:/3D-CNN/D/All_Models/3D-CNN_LSTM_Adv_weights')

In [17]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [29]:
class switch_1():
    def index_threshhold(conf, classes , pre):
        if classe == 0:
            print(conf[0])
            if pre == 0:
                return(classe)
            else:
                return(5)
        elif classe == 1:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe ==  2 :
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 3:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 4:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 5:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 6:
                return(6)
    def puttext_on(conf, index, classes,frame, font):   
        cv2.putText(frame, 'confidence: ' + str(conf), (10, 90), font, 0.7, (56, 71, 255), 2, 1)
        cv2.putText(frame, 'gesture class: ' + classe , (10, 60), font, 0.7, (56, 71, 255), 2, 1)

In [38]:
cap.release()
cv2.destroyAllWindows()

In [20]:
# Control for MediaPlayer:

import time
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
pTime = 0
cTime = 0
cap_region_x_begin = 0.5  
cap_region_y_end = 0.8  
threshold = 60  
blurValue = 41 
bgSubThreshold = 50
learningRate = 0
pre = 0
conf = 0

segmentor = SelfiSegmentation()
classe =''
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
    fgmask = bgModel.apply(frame)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold( gray,60,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    to_predict.append(cv2.resize(thresh, (64, 64)))
    
    
        
    if len(to_predict) == 30:
        frame_to_predict = np.array(to_predict, dtype=np.float32)
        frame_to_predict = normaliz_data(frame_to_predict)
        result_1 = model1.predict(frame_to_predict)
        result_2 = model2.predict(frame_to_predict)
        result_3 = model3.predict(frame_to_predict)
        result_4 = model4.predict(frame_to_predict)
        
        # print(result)
        classe = classes[np.argmax(result_1+result_2)]
        predict = np.amax((result_1+result_2)/2)
        print(classe,(predict)*100,'%')
        to_predict = []
        
        real_index = switch_1.index_threshhold(predict, classe ,pre)
        # instruction = switch_1.controll_PC(real_index)
        pre = classes[0]
        switch_1.puttext_on(predict, real_index, classes[6], frame, font)
        cv2.putText(frame, 'FPS: ' + str(int(fps)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (56, 71, 255), 2, 1)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.imshow('Hand Gesture Recognition',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release
cv2.destroyAllWindows()

Stop Sign 70.99746465682983 %
Zooming In With Two Fingers 49.422723054885864 %
Stop Sign 50.00144839286804 %
Stop Sign 74.96974468231201 %
Zooming Out With Two Fingers 49.906980991363525 %
Zooming In With Two Fingers 79.70004081726074 %
Doing other things 61.588019132614136 %
Swiping Right 77.85154581069946 %
Swiping Left 96.96136713027954 %
Doing other things 50.11880397796631 %
Swiping Right 71.27473950386047 %
Swiping Right 54.14236783981323 %
Doing other things 50.00724792480469 %
Swiping Right 99.77268576622009 %
Doing other things 94.13763880729675 %
Swiping Right 66.14055633544922 %
Swiping Right 52.0447313785553 %
Doing other things 85.13463139533997 %
Doing other things 85.1012110710144 %
Stop Sign 73.42141270637512 %
Doing other things 49.99447166919708 %
Doing other things 99.98056888580322 %
Stop Sign 49.53074753284454 %
Zooming Out With Two Fingers 99.75488185882568 %
Zooming Out With Two Fingers 97.9312539100647 %
Zooming In With Two Fingers 38.03446590900421 %
Zooming Ou

In [24]:
# Control for PPT:

import time
import pyautogui
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
pTime = 0
cTime = 0
cap_region_x_begin = 0.5  
cap_region_y_end = 0.8  
threshold = 60  
blurValue = 41 
bgSubThreshold = 50
learningRate = 0
pre = 0
conf = 0

segmentor = SelfiSegmentation()
classe =''
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
    fgmask = bgModel.apply(frame)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    to_predict.append(cv2.resize(gray, (64, 64)))
    
    
        
    if len(to_predict) == 30:
        frame_to_predict = np.array(to_predict, dtype=np.float32)
        frame_to_predict = normaliz_data(frame_to_predict)
        result_1 = model1.predict(frame_to_predict)
        result_2 = model2.predict(frame_to_predict)
        result_3 = model3.predict(frame_to_predict)
        result_4 = model4.predict(frame_to_predict)
        
        # print(result)
        classe = classes[np.argmax(result_1+result_2+result_3+result_4)]
        predict = np.amax((result_1+result_2+result_3+result_4)/4)
        print(classe,(predict)*100,'%')
        if(classe == "Swiping Left"):
                pyautogui.press('left', presses = 1)

        elif(classe == "Swiping Right"):
                pyautogui.press('right', presses = 1)

        elif(classe == "Zooming In With Two Fingers"):
                pyautogui.hotkey('shift','f5')
                
        elif(classe == "Zooming Out With Two Fingers"):
                pyautogui.press('esc')
                
        elif(classe == "Doing other things"):
                pyautogui.press('None')

        else:
            pyautogui.press('None')

        to_predict = []
        real_index = switch_1.index_threshhold(predict, classe ,pre)
        # instruction = switch_1.controll_PC(real_index)
        pre = classes[0]
    switch_1.puttext_on(predict, real_index, classes[6], frame, font)
    cv2.putText(frame, 'FPS: ' + str(int(fps)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (56, 71, 255), 2, 1)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime    
    cv2.imshow('Hand Gesture Recognition',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release
cv2.destroyAllWindows()

Doing other things 63.58610987663269 %
Swiping Right 45.21037042140961 %
Swiping Right 35.71683168411255 %
No gesture 49.10386800765991 %
Doing other things 78.01382541656494 %
Swiping Right 50.07385015487671 %
No gesture 49.98246431350708 %
Doing other things 49.99953508377075 %
Swiping Right 45.21418213844299 %
Swiping Right 72.07341194152832 %
Doing other things 50.308626890182495 %
Doing other things 49.97339844703674 %
Zooming In With Two Fingers 47.8986918926239 %
Doing other things 49.548304080963135 %
Zooming In With Two Fingers 49.63546395301819 %
Doing other things 48.723167181015015 %
Stop Sign 50.225937366485596 %
Stop Sign 49.99947547912598 %
Stop Sign 49.999237060546875 %
Stop Sign 49.89791512489319 %
No gesture 47.9771614074707 %
Swiping Right 57.67450928688049 %
Zooming Out With Two Fingers 42.882394790649414 %
Doing other things 58.24919939041138 %
Stop Sign 51.73928141593933 %
Swiping Right 55.550140142440796 %
Stop Sign 99.93307590484619 %
Stop Sign 49.48423206806183

KeyboardInterrupt: 

In [34]:
# Control for MediaPlayer:

import time
import pyautogui
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
pTime = 0
cTime = 0
cap_region_x_begin = 0.5  
cap_region_y_end = 0.8  
threshold = 60  
blurValue = 41 
bgSubThreshold = 50
learningRate = 0
pre = 0
conf = 0
segmentor = SelfiSegmentation()
classe =''
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
    fgmask = bgModel.apply(frame)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray,60,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    to_predict.append(cv2.resize(thresh, (64, 64)))
    
    
        
    if len(to_predict) == 30:
        frame_to_predict = np.array(to_predict, dtype=np.float32)
        frame_to_predict = normaliz_data(frame_to_predict)
        result_1 = model1.predict(frame_to_predict)
        result_2 = model2.predict(frame_to_predict)
        result_3 = model3.predict(frame_to_predict)
        result_4 = model4.predict(frame_to_predict)
        
        # print(result)
        classe = classes[np.argmax(result_1+result_2+result_3+result_4)]
        predict = np.amax((result_1+result_2+result_3+result_4)/4)
        print(classe,(predict)*100,'%')
        if(classe == "Swiping Left"):
                pyautogui.press('left', presses = 4)

        elif(classe == "Swiping Right"):
                pyautogui.press('right', presses = 4)

        elif(classe == "Zooming In With Two Fingers"):
                pyautogui.hotkey('up', presses = 20)
                
        elif(classe == "Zooming Out With Two Fingers"):
                pyautogui.press('down', presses = 20)
        
        elif(classe == "Stop Sign"):
                pyautogui.press('space', presses = 1)
                
        elif(classe == "Doing other things"):
                pyautogui.press('None')

        else:
            pyautogui.press('None')

        to_predict = []

        real_index = switch_1.index_threshhold(predict, classe ,pre)
        # instruction = switch_1.controll_PC(real_index)
        pre = classes[0]
    switch_1.puttext_on(predict, real_index, classes[6], frame, font)
    cv2.putText(frame, 'FPS: ' + str(int(fps)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (56, 71, 255), 2, 1)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime   
    cv2.imshow('Hand Gesture Recognition',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release
cv2.destroyAllWindows()

Doing other things 64.4906222820282 %
Stop Sign 48.61963987350464 %
Doing other things 81.11651539802551 %
Doing other things 99.61621761322021 %
Stop Sign 93.46321225166321 %
Doing other things 90.16807079315186 %
Doing other things 99.99983310699463 %
Doing other things 99.99732971191406 %
Doing other things 99.99738931655884 %
Doing other things 50.10256767272949 %
Doing other things 99.99989867210388 %
Doing other things 99.99884366989136 %
Doing other things 68.36144924163818 %
Doing other things 95.08786201477051 %
Doing other things 76.2931227684021 %
Doing other things 99.99980330467224 %
Doing other things 99.94388818740845 %
No gesture 50.00552535057068 %
Doing other things 81.64834976196289 %
Doing other things 99.28104877471924 %
Doing other things 59.62544083595276 %
Doing other things 99.99404549598694 %
Doing other things 99.3706464767456 %
Swiping Left 99.92018938064575 %
No gesture 72.201669216156 %
Doing other things 99.91393089294434 %
Doing other things 99.999976158

In [37]:
# Control for Youtube:

import time
import pyautogui
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
pTime = 0
cTime = 0
cap_region_x_begin = 0.5  
cap_region_y_end = 0.8  
threshold = 60  
blurValue = 41 
bgSubThreshold = 50
learningRate = 0
pre = 0
conf = 0
segmentor = SelfiSegmentation()
classe =''
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
    fgmask = bgModel.apply(frame)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray,60,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    to_predict.append(cv2.resize(thresh, (64, 64)))
    
    
        
    if len(to_predict) == 30:
        frame_to_predict = np.array(to_predict, dtype=np.float32)
        frame_to_predict = normaliz_data(frame_to_predict)
        result_1 = model1.predict(frame_to_predict)
        result_2 = model2.predict(frame_to_predict)
        result_3 = model3.predict(frame_to_predict)
        result_4 = model4.predict(frame_to_predict)
        
        # print(result)
        classe = classes[np.argmax(result_1+result_2+result_3+result_4)]
        predict = np.amax((result_1+result_2+result_3+result_4)/4)
        print(classe,(predict)*100,'%')
        if(classe == "Swiping Left"):
                pyautogui.hotkey('shift','P')

        elif(classe == "Swiping Right"):
                pyautogui.hotkey('shift','N')

        elif(classe == "Zooming In With Two Fingers"):
                pyautogui.press('f')
                
        elif(classe == "Zooming Out With Two Fingers"):
                pyautogui.press('esc')
                
        elif(classe == "Stop Sign"):
                pyautogui.press('space', presses = 1)
                
        elif(classe == "Doing other things"):
                pyautogui.press('None')

        else:
            pyautogui.press('None')

        to_predict = []

        real_index = switch_1.index_threshhold(predict, classe ,pre)
        # instruction = switch_1.controll_PC(real_index)
        pre = classes[0]
        switch_1.puttext_on(predict, real_index, classes[6], frame, font)
        cv2.putText(frame, 'FPS: ' + str(int(fps)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (56, 71, 255), 2, 1)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime   
    cv2.imshow('Hand Gesture Recognition',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release
cv2.destroyAllWindows()

Doing other things 49.989140033721924 %
Doing other things 98.28996658325195 %
Doing other things 99.63550567626953 %
No gesture 49.869486689567566 %
Doing other things 55.33878207206726 %
Doing other things 99.99374151229858 %
Zooming In With Two Fingers 50.11935830116272 %
Doing other things 50.0000536441803 %
Swiping Right 50.79832077026367 %
Doing other things 98.43630194664001 %
Doing other things 97.36584424972534 %
Swiping Right 49.0287721157074 %
Doing other things 99.99743700027466 %
Doing other things 99.53892230987549 %
Doing other things 65.23011326789856 %
Doing other things 99.20092821121216 %
Doing other things 100.0 %
Doing other things 99.07864332199097 %
Doing other things 97.18233346939087 %
Doing other things 99.99586343765259 %
Swiping Right 48.5540509223938 %
Doing other things 63.872748613357544 %
Doing other things 93.22965145111084 %
Doing other things 99.92541074752808 %
Doing other things 78.25020551681519 %
Doing other things 99.67454671859741 %
Doing other 